In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/SP500_EGS_Score_avarage_per_year.csv', index_col=0, parse_dates=True,sep=',')

/var/folders/q4/1vtck34d7h12n065tcl6nwkm0000gn/T/ipykernel_31805/1195792824.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../../data/SP500_EGS_Score_avarage_per_year.csv', index_col=0, parse_dates=True,sep=',')


In [15]:
df.head(20)

,year,Company_Symbol,Total-Score,E-Score,S-Score,G-Score
0,2014,A,72.50,83.00,73.00,63.00
1,2014,AAL,72.75,77.00,67.00,75.25
2,2014,AAPL,60.75,74.00,45.00,62.00
3,2014,ABBV,64.50,70.25,60.25,67.50
4,2014,ABT,64.00,71.00,57.00,69.00
5,2014,ACGL,46.00,34.00,54.00,47.50
6,2014,ACN,72.00,77.00,66.00,78.00
7,2014,ADBE,70.00,75.00,71.00,63.00
8,2014,ADI,65.50,64.75,66.00,66.00
9,2014,ADM,52.25,56.50,41.75,62.50


In [16]:
# all lower case

df.columns = map(str.lower, df.columns)

# - to _

df.columns = df.columns.str.replace('-', '_')

In [21]:
len(df.company_symbol.unique())


441

In [2]:
import sys 
sys.path.append('../')

In [4]:
from helpers.text_preprocessing.preprocess_text import preprocess_text

In [5]:
import pandas as pd

# Load or create your DataFrame (replace 'your_data.csv' with the path to your data file)
df = pd.read_csv('../data/extracted_text_sustainability_reports.csv', index_col=0, parse_dates=True,sep=',')


/var/folders/t5/tjfw6fh92wvd7nqk0dkpf15h0000gn/T/ipykernel_40193/1936642105.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../data/extracted_text_sustainability_reports.csv', index_col=0, parse_dates=True,sep=',')


In [9]:
# Preprocess the text and get the preprocessed DataFrame
preprocessed_df = preprocess_text(df)


KeyboardInterrupt: 